In [1]:
import os
os.getcwd()

'C:\\Users\\jerom'

In [3]:
!pip install openai langchain langchain-openai \
email-validator python-dotenv mailjet_rest --quiet


In [4]:
import requests, pprint

#URL complète avec la clé API et les coordonnées ici la clé API n'est pas sensible (gratuite, et de toute façon on travaille en GET)
url = "http://api.weatherapi.com/v1/current.json?key=98adae444b4947a3a4e91950250905&q=43.937,4.813&aqi=no"

# Requête GET
response = requests.get(url)

# Vérification
if response.status_code == 200:
    weather_json = response.json()
    print("✅ Données récupérées avec succès")
else:
    print(f"❌ Erreur {response.status_code} lors de la requête")

pprint.pprint(weather_json)

def format_meteo_phrase(weather_json):
    # Extraction des données
    cloud = weather_json["current"]["cloud"]
    precip = weather_json["current"]["precip_mm"]
    temp = weather_json["current"]["temp_c"]
    uv = weather_json["current"]["uv"]
    wind = weather_json["current"]["wind_kph"]

    # Construction de la phrase
    phrase = (
        f"Couverture nuageuse : {cloud}%, "
        f"Pluies : {precip} mm, "
        f"Température : {temp} degrés Celsius, "
        f"Indice UV : {uv}, "
        f"Vent : {wind} km/h"
    )

    return phrase

# Génération de la phrase météo
meteo_phrase = format_meteo_phrase(weather_json)
print(meteo_phrase)

✅ Données récupérées avec succès
{'current': {'cloud': 0,
             'precip_mm': 0.0,
             'temp_c': 25.3,
             'uv': 1.5,
             'wind_kph': 9.0},
 'location': {'country': 'France',
              'lat': 43.95,
              'localtime': '2025-08-05 09:57',
              'localtime_epoch': 1754380632,
              'lon': 4.817,
              'name': 'Avignon',
              'region': "Provence-Alpes-Cote d'Azur",
              'tz_id': 'Europe/Paris'}}
Couverture nuageuse : 0%, Pluies : 0.0 mm, Température : 25.3 degrés Celsius, Indice UV : 1.5, Vent : 9.0 km/h


In [10]:
import os
from dotenv import load_dotenv

# Chargement explicite du fichier
load_dotenv(dotenv_path="ENV/env.txt")

# Récupération propre de la clé
openai.api_key = os.getenv("OPENAI_API_KEY")

# Vérification
assert openai.api_key is not None, "❌ Clé API non trouvée dans ENV/env.txt"
print("✅ Clé API chargée avec succès")



✅ Clé API chargée avec succès


In [12]:
# Extraction des variables
location = weather_json["location"]["name"]
temp = weather_json["current"]["temp_c"]
wind = weather_json["current"]["wind_kph"]
precip = weather_json["current"]["precip_mm"]
cloud = weather_json["current"]["cloud"]
uv = weather_json["current"]["uv"]
time = weather_json["location"]["localtime"]

# Appel à OpenAI avec prompt structuré
client = openai.OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "Tu es une IA experte en météorologie. Tu génères un bulletin météo professionnel, clair, synthétique et compréhensible, à partir des données brutes fournies. Mentionne la ville, la date, et les conditions météorologiques de manière fluide. Ne répète pas les unités si elles sont déjà claires dans la phrase."
        },
        {
            "role": "user",
            "content": f"""
Voici les données météo à utiliser :

- Lieu : {location}
- Date et heure locale : {time}
- Température : {temp} °C
- Vent : {wind} km/h
- Précipitations : {precip} mm
- Couverture nuageuse : {cloud} %
- Indice UV : {uv}

Peux-tu rédiger un bulletin météo court et professionnel ?
"""
        }
    ]
)

# Affichage du résultat
bulletin = response.choices[0].message.content
print("📄 Bulletin météo généré :\n")
print(bulletin)

📄 Bulletin météo généré :

Bienvenue à Avignon pour ce lundi 5 août 2025. Actuellement, la température est de 25.3 °C avec un vent de 9.0 km/h. Aucune précipitation n'est prévue avec une couverture nuageuse de 0 %. L'indice UV est de 1.5, veillez à rester protégé du soleil si vous sortez.


In [13]:
# --- Vérification adresse e-mail ---
import email_validator
from mailjet_rest import Client

# Récupération des variables d’environnement

load_dotenv(dotenv_path="ENV/env.txt")

mj_api_key = os.getenv("MJ_API_KEY_PUBLIC")
mj_api_secret = os.getenv("MJ_API_KEY_PRIVATE")
expediteur = os.getenv("MJ_FROM_EMAIL")  # ← à ajouter dans env.txt
destinataire = os.getenv("MJ_TO_EMAIL")  # ← à ajouter dans env.txt

# Validation des adresses email
for email in [expediteur, destinataire]:
    try:
        email_validator.validate_email(email)
    except email_validator.EmailNotValidError as e:
        raise ValueError(f"Adresse email non valide : {email} → {e}")

# Création du client Mailjet
mj = Client(auth=(mj_api_key, mj_api_secret), version='v3.1')

# Envoi du mail
data = {
    'Messages': [
        {
            "From": {
                "Email": expediteur,
                "Name": "Bulletin Météo Automatique"
            },
            "To": [{"Email": destinataire}],
            "Subject": f"Bulletin météo – {location} – {time.split(' ')[0]}",
            "TextPart": bulletin,
            "HTMLPart": f"<h3>Bulletin météo – {location}</h3><p>{bulletin.replace(chr(10), '<br>')}</p>"
        }
    ]
}

result = mj.send.create(data=data)

if result.status_code == 200:
    print("✅ Email envoyé avec succès")
else:
    print(f"❌ Échec envoi : {result.status_code} / {result.json()}")


✅ Email envoyé avec succès
